In [1]:
import transformers
import huggingface_hub
from huggingface_hub import notebook_login
import ipywidgets

In [2]:
notebook_login()

In [66]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader  
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 

In [9]:
data = pd.read_csv("data/tweet_text_2.csv", encoding = 'cp1252') 
tweet_text = data['TWEET_TEXT']
tweet_lengths = [len(tweet) for tweet in tweet_text]


Finetuning BERT to see performance for predicting gasoline shortage

In [6]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizing data and creating trainign an test sets 

In [10]:
#def get_tokenized_data(row):
#    input_ids, token_type_ids, attention_masks = tokenizer(row["TWEET_TEXT"], padding = "max_length", truncation = True)
#    row['input_ids'] = input_ids["input_ids"]
#    row['token_type_ids'] = token_type_ids["input_ids"]
#    row['attention_masks'] = attention_masks[""]
#    return row


#data = data.apply(lambda row: get_tokenized_data(row), axis =1)
data = data[data.Shortage.notna()]
train_data = data.sample(frac =0.8)
test_data = data[~data.index.isin(train_data.index)]

In [8]:
train_data

,SNO,TWEET_TEXT,Shortage,Demand,Probable Shortage,input_ids,token_type_ids,attention_masks
2205,305773,I never thought I would have to drive around 3...,1.0,1.0,1.0,input_ids,token_type_ids,attention_mask
222,48552,Time to try and find gas,1.0,1.0,1.0,input_ids,token_type_ids,attention_mask
1138,147005,I miss driving my cars and yelling at idiot dr...,0.0,1.0,0.0,input_ids,token_type_ids,attention_mask
3018,696130,Viewer Mike Ross sent me this picture of the S...,1.0,1.0,1.0,input_ids,token_type_ids,attention_mask
2927,597353,I swea yall gone feel me this year . All gas n...,0.0,0.0,0.0,input_ids,token_type_ids,attention_mask
...,...,...,...,...,...,...,...,...
1070,140697,Consider using local roads if you're heading n...,0.0,1.0,1.0,input_ids,token_type_ids,attention_mask
3123,727748,I need some gas like yesterday,0.0,1.0,0.0,input_ids,token_type_ids,attention_mask
800,109299,Y'all say it's hard to find gas? I literally d...,0.0,1.0,1.0,input_ids,token_type_ids,attention_mask
2660,426291,I stop at every gas stations here and none of ...,0.0,0.0,0.0,input_ids,token_type_ids,attention_mask


explpring the layers of bert base case

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [60]:
class Model(nn.Module):
        
    def __init__(self, num_labels):
        super().__init__()
        self.bert = model
        self.linear = nn.Linear(768,num_labels)
        self.softmax = nn.Softmax(1)
    def forward(self,token_ids, attention_masks ):
        _,bert_output = model(token_ids, attention_mask=attention_masks,return_dict=False)
        output = self.softmax(self.linear(bert_output))
        return output
   
num_labels = 2    
finetune_model = Model(num_labels)



Will finetune only the last layer in the first model
MODEL 1 

In [61]:
for params in finetune_model.parameters():
    params.requires_grad = False
    
for params in finetune_model.linear.parameters():
    params.requires_grad = True


Setting parameters, optimizer

In [62]:
batch_size = 32
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-5
optimizer = torch.optim.Adam(finetune_model.parameters(), lr=learning_rate)

Creating a dataset and dataloader

In [42]:
class CustomDataSet(Dataset):
    def __init__(self, token_ids, attention_masks, labels):
        self.tokens = token_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.tokens[idx], self.attention_masks[idx], self.labels[idx]
    
train_sample = tokenizer.batch_encode_plus(list(train_data["TWEET_TEXT"]), padding = "max_length", truncation=True)
test_sample = tokenizer.batch_encode_plus(list(test_data["TWEET_TEXT"]), padding = "max_length", truncation=True)
train_dataset = CustomDataSet(torch.tensor(train_sample["input_ids"]), torch.tensor(train_sample["attention_mask"]), torch.tensor(list(train_data['Shortage'])))
test_dataset = CustomDataSet(torch.tensor(test_sample["input_ids"]), torch.tensor(test_sample["attention_mask"]), torch.tensor(list(test_data['Shortage'])))
#test_dataset = CustomDataSet(list(test_data["tokenized_tweets"]), list(test_data['Shortage']))
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size)


In [27]:
type(train_sample["attention_mask"])

list

In [81]:
loss_adam = []
n_epochs = 10

for epoch in range(n_epochs):
    epoch_loss_train, epoch_loss_test, epoch_accuracy_train, epoch_acccuracy_test = 0, 0, 0, 0
    outputs_train, outputs_test, labels_epochs_train, labels_epochs_test  = [], [], [], []
    n = 0 
    for step, batch in enumerate(train_dataloader): 
        n += 1
        batch_loss = 0   
        input_ids, attention_masks, labels = batch
        output = finetune_model(input_ids, attention_masks)
        loss = criterion(output, labels.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"epoch/total epoch : {epoch}/{n_epochs}---- step:{step}----- loss:{loss.item()}")
        epoch_loss_train += loss.item() 
        output = output.detach().numpy()
        labels = output.detach().numpy()
        outputs_train.append(output)
        labels_epochs_train = labels_epochs_train.append(labels)
    outputs_train = np.concatenate(outputs_train, axis =1)
    label_epochs_train = np.concatenate(label_epochs_train, axis =1)
    epoch_accuracy_train = sum(outputs_train == label_epochs_train)/(outputs_train.shape[0])
    for step, batch in enumerate(test_dataloader):
        input_ids, attention_masks, labels = batch
        output = finetune_model(input_ids, attention_masks)
        loss = criterion(output, labels.long())
        epoch_loss_test += loss.item()
        output = output.detach().numpy()
        labels = output.detach().numpy()
    outputs_test = np.concatenate(outputs_test, axis =1)
    label_epochs_test = np.concatenate(label_epochs_test, axis =1)
    epoch_accuracy_test = sum(outputs_test == label_epochs_test)/(outputs_test.shape[0])
    print(f"epoch/total epoch : {epoch}/{n_epochs} --- train loss:{epoch_loss_train/n} --- test loss:{epoch_loss_test/n} --- train accuracy:{epoch_accuracy_train} --- test accuracy:{epoch_accuracy_test}")







: 

In [80]:
a  = np.array([0,1,0,2])
b = np.array([0,1,1,2])

sum(a == b)/a.shape[0]

np.float64(0.75)